In [ ]:
!nvidia-smi


In [ ]:
import cv2
import os
import time
import csv
import mimetypes
import torch
import numpy as np

from enum import Enum
from pathlib import Path
from tqdm.auto import tqdm
from abc import ABCMeta, abstractmethod


In [ ]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
DEVICE


In [ ]:
from ultralytics import YOLO
from depth_anything_v2.dpt import DepthAnythingV2


In [ ]:
model_configs = {
    'vits': {'encoder': 'vits', 'features': 64, 'out_channels': [48, 96, 192, 384]},
    'vitb': {'encoder': 'vitb', 'features': 128, 'out_channels': [96, 192, 384, 768]},
    'vitl': {'encoder': 'vitl', 'features': 256, 'out_channels': [256, 512, 1024, 1024]},
    'vitg': {'encoder': 'vitg', 'features': 384, 'out_channels': [1536, 1536, 1536, 1536]}
}


In [ ]:
yolo = YOLO('./weight_all_class/weights/best.pt')
depth_anything = DepthAnythingV2(**model_configs['vits']).to(DEVICE).eval()


In [ ]:
depth_anything.load_state_dict(torch.load(f'checkpoints/depth_anything_v2_{"vits"}.pth', map_location='cpu'))
depth_anything = depth_anything.to(DEVICE).eval()


In [ ]:
img = cv2.imread('./test_img/Untitled.png')


In [ ]:
results = yolo.predict(
    source=img, save=True
)


In [ ]:
depth = depth_anything.infer_image(img)
depth = (depth - depth.min()) / (depth.max() - depth.min()) * 255.0
depth = depth.astype(np.uint8)


In [ ]:
r_dict = dict()

for r in results:
    r_dict = {idx: list(map(int, bbox)) for idx, bbox in enumerate(r.boxes.xyxy.squeeze().tolist())}


In [ ]:
r_dict


In [ ]:
background = np.zeros((img.shape[0], img.shape[1]), dtype=np.uint8)
background


In [ ]:
import matplotlib.pyplot as plt


def plotImageRGB(image: list):
    plt.imshow(image)
    plt.axis(False)


def plotImageGRAY(image: list):
    plt.imshow(image, cmap='gray')
    plt.axis(False)


In [ ]:
plotImageRGB(img)


In [ ]:
plotImageGRAY(depth)


In [ ]:
plt.imshow(background, cmap='gray')
plt.axis(False)


In [ ]:
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)


In [ ]:
cv2.rectangle(background, (r_dict[0][:2]), (r_dict[0][2:]), 255, -1)
plt.imshow(background, cmap='gray')
plt.axis(False)


In [ ]:
fish_depth = cv2.bitwise_and(depth, depth, mask=background)
plotImageGRAY(fish_depth)


In [ ]:
_, otsu_depth = cv2.threshold(fish_depth, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
plotImageGRAY(otsu_depth)


In [ ]:
edges = cv2.Canny(otsu_depth, 0, 255)
edges


In [ ]:
plt.imshow(edges, cmap='gray')
plt.axis(False)


In [ ]:
contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
edge = max(contours, key=cv2.contourArea)
background = np.zeros((img.shape[0], img.shape[1]), dtype=np.uint8)


In [ ]:
len(edges), type(edges)


cv2.fillPoly(background, pts=[edges[0]], color=(255, 255, 255))


In [ ]:
cv2.drawContours(background, [edge], -1, (255, 255, 255), thickness=cv2.FILLED)


In [ ]:
plotImageGRAY(background)


In [ ]:
from sklearn.decomposition import PCA

# 4. 컨투어 좌표를 추출하고 PCA 수행

# 컨투어의 좌표를 (x, y) 배열로 변환
contour_points = np.squeeze(edge)

# PCA 적용
pca = PCA(n_components=2)
pca.fit(contour_points)

# 주성분 축을 시각화하기 위한 계산
center = np.mean(contour_points, axis=0)
eigenvectors = pca.components_  # 주성분 방향
eigenvalues = pca.explained_variance_  # 주성분 크기

# 결과 시각화를 위한 출력 이미지 생성
output_image = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)

# 5. 컨투어 그리기
cv2.drawContours(output_image, [edge], -1, (0, 255, 0), 2)

# 6. 주성분 축 시각화
scale = 100  # 주성분 축을 시각적으로 확장하는 스케일링
for eigenvalue, eigenvector in zip(eigenvalues, eigenvectors):
    # 주성분 축의 시작과 끝 좌표 계산
    endpoint = center + eigenvector * np.sqrt(eigenvalue) * scale

    # 주성분 축 그리기 (파란색 선)
    cv2.line(output_image, tuple(center.astype(int)), tuple(endpoint.astype(int)), (255, 0, 0), 2)

# 결과 시각화
plt.figure(figsize=(10, 10))
plt.imshow(cv2.cvtColor(output_image, cv2.COLOR_BGR2RGB))
plt.title('PCA on Contours'), plt.axis('off')
plt.show()


In [ ]:
import numpy as np
import cv2
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt


def find_tail_split_point(contour):
    """
    컨투어에서 두 주성분에 가장 가까운 점을 찾아 물고기 꼬리의 갈라진 중앙점을 반환하는 함수
    """
    # 1. 컨투어 좌표 추출 (N x 2 배열)
    contour_points = np.squeeze(contour)

    # PCA 적용 (주성분 분석)
    pca = PCA(n_components=2)
    pca.fit(contour_points)

    # 주성분 축
    center = np.mean(contour_points, axis=0)  # 중심점
    eigenvectors = pca.components_  # 주성분 벡터 (2개의 주성분)
    eigenvalues = pca.explained_variance_  # 주성분에 대한 분산 (크기)

    # 2. 주성분 벡터로부터 컨투어의 각 점까지의 거리 계산
    def point_distance_to_line(point, line_vector, line_point):
        # 선분과 점 사이의 직선 거리를 계산
        point = np.array(point)
        line_vector = np.array(line_vector)
        line_point = np.array(line_point)

        # 두 벡터 간의 크로스 프로덕트를 사용해 거리를 계산
        distance = np.linalg.norm(np.cross(line_vector, line_point - point)) / np.linalg.norm(line_vector)
        return distance

    # 첫 번째 주성분과 두 번째 주성분에 대한 거리 계산
    distances_to_first_pc = [point_distance_to_line(p, eigenvectors[0], center) for p in contour_points]
    distances_to_second_pc = [point_distance_to_line(p, eigenvectors[1], center) for p in contour_points]

    # 3. 첫 번째 주성분에 대해 먼저 정렬, 그 후 두 번째 주성분 기준으로 추가 정렬
    sorted_points = sorted(contour_points, key=lambda p: (
        point_distance_to_line(p, eigenvectors[0], center),  # 첫 번째 주성분 기준 정렬
        point_distance_to_line(p, eigenvectors[1], center)   # 두 번째 주성분 기준 추가 정렬
    ), )

    # 첫 번째 주성분과 두 번째 주성분과 가장 가까운 점 선택
    closest_point = sorted_points[1]

    return closest_point, center, eigenvectors, sorted_points


# 가장 큰 컨투어 선택
contour = max(contours, key=cv2.contourArea)

# 함수 호출하여 꼬리 중앙점 찾기
tail_split_point, center, eigenvectors, sorted_points = find_tail_split_point(contour)

# 시각화
output_image = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
cv2.drawContours(output_image, [contour], -1, (0, 255, 0), 2)  # 컨투어 그리기
cv2.circle(output_image, tuple(tail_split_point.astype(int)), 5, (0, 0, 255), -1)  # 꼬리 중앙점 표시 (빨간색)

# 주성분 축 그리기
scale = 100
for eigenvalue, eigenvector in zip(eigenvectors, eigenvectors):
    endpoint = center + eigenvector * scale
    cv2.line(output_image, tuple(center.astype(int)), tuple(endpoint.astype(int)), (255, 0, 0), 2)  # 파란색 주성분 축

# 결과 시각화
plt.figure(figsize=(10, 10))
plt.imshow(cv2.cvtColor(output_image, cv2.COLOR_BGR2RGB))
plt.title('Fish Tail Split Point Detection'), plt.axis('off')
plt.show()


In [ ]:
# OpenCV에서 제공하는 다양한 커널을 사용하여 erode 연산을 적용하는 예시를 보여줍니다.
kernels = {
    "Rectangular (5x5)": cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5)),
    "Elliptical (5x5)": cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5)),
    "Cross-shaped (5x5)": cv2.getStructuringElement(cv2.MORPH_CROSS, (5, 5))
}


In [ ]:
# 결과 시각화를 위한 준비
plt.figure(figsize=(15, 5))

# 원본 이미지 시각화
plt.subplot(1, 4, 1)
plt.imshow(background, cmap='gray')
plt.title('Original Binary Image')
plt.axis('off')

# 각 커널에 대해 erode 연산을 적용하고 시각화
for i, (name, kernel) in enumerate(kernels.items(), start=2):
    eroded_image = cv2.erode(background, kernel, iterations=5)

    # 시각화
    plt.subplot(1, 4, i)
    plt.imshow(eroded_image, cmap='gray')
    plt.title(name)
    plt.axis('off')

plt.show()


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN


def preprocess_image(image_path):
    image = cv2.imread(image_path)
    if image is None:
        print("이미지를 불러올 수 없습니다.")
        return None

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    thresh = cv2.adaptiveThreshold(
        blurred, 255,
        cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
        cv2.THRESH_BINARY_INV,
        11, 2
    )
    return thresh


def detect_edges(thresh_image):
    edges = cv2.Canny(thresh_image, 50, 150, apertureSize=3)
    return edges


def detect_lines(edges):
    lines = cv2.HoughLinesP(
        edges,
        rho=1,
        theta=np.pi / 180,
        threshold=50,
        minLineLength=50,
        maxLineGap=10
    )
    return lines


def draw_lines(image, lines):
    line_img = image.copy()
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            cv2.line(line_img, (x1, y1), (x2, y2), (0, 255, 0), 2)
    return line_img


def find_split_points(lines):
    if lines is None:
        return []

    start_points = [(line[0][0], line[0][1]) for line in lines]

    clustering = DBSCAN(eps=10, min_samples=2).fit(start_points)
    labels = clustering.labels_

    split_points = []
    for label in set(labels):
        if label == -1:
            continue
        points = np.array(start_points)[labels == label]
        centroid = points.mean(axis=0).astype(int)
        split_points.append(tuple(centroid))

    return split_points


def visualize_results(original_image, edges, line_image, split_points):
    plt.figure(figsize=(20, 10))

    plt.subplot(2, 2, 1)
    plt.imshow(cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB))
    plt.title('Original Image')
    plt.axis('off')

    plt.subplot(2, 2, 2)
    plt.imshow(edges, cmap='gray')
    plt.title('Edge Detection')
    plt.axis('off')

    plt.subplot(2, 2, 3)
    plt.imshow(cv2.cvtColor(line_image, cv2.COLOR_BGR2RGB))
    plt.title('Detected Lines')
    plt.axis('off')

    plt.subplot(2, 2, 4)
    plt.imshow(cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB))
    for point in split_points:
        plt.scatter(point[0], point[1], c='red', s=100, marker='x')
    plt.title('Split Points')
    plt.axis('off')

    plt.show()


def main(image_path):
    thresh = preprocess_image(image_path)
    if thresh is None:
        return

    edges = detect_edges(thresh)
    lines = detect_lines(edges)
    original = cv2.imread(image_path)
    line_image = draw_lines(original, lines)
    split_points = find_split_points(lines)
    visualize_results(original, edges, line_image, split_points)

    print("찾은 갈라진 지점:", split_points)


In [ ]:
main('./test_img/Untitled.png')
